# Obtenção de rating do contribuinte

Com o objetivo de direcionar o modelo que ditá quais dívidas são melhores de recuperar, será realizado um rating que envolva apenas as variáveis do contribuinte.

In [95]:
import os
import dotenv
import zipfile
import pandas as pd
import numpy as np
from datetime import date

from sklearn.cluster import KMeans
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.preprocessing import OneHotEncoder

import seaborn as sns
from matplotlib import pyplot as plt
import plotly.graph_objects as go

import pickle
import shutil

In [96]:
import warnings
warnings.filterwarnings("ignore")

In [97]:
rootPath = os.getcwd()
dataPath = os.path.join(rootPath, 'data')
modelsPath = os.path.join(rootPath, 'models')
env = os.path.join(rootPath, '.env')
dotenv.load_dotenv(dotenv_path=env)

True

In [98]:
import locale

locale.setlocale(locale.LC_ALL, 'pt_BR.UTF-8')


def formatar_moeda(valor):
    return locale.currency(valor, grouping=True)

# 01) Importando dados

In [99]:
zip_file = os.path.join(dataPath, 'base_treino.zip')
z = zipfile.ZipFile(zip_file)

In [100]:
def ler_bases_exportadas(nome_arquivo):
    z.extract(nome_arquivo)
    df = pd.read_csv(nome_arquivo, sep=',')
    os.remove(nome_arquivo)
    return df

In [101]:
base_conjunta = ler_bases_exportadas('imovel_mercantil.csv')
# base_imovel = ler_bases_exportadas('imovel.csv')
# base_mercantil = ler_bases_exportadas('mercantil.csv')
base_notas_fiscais = ler_bases_exportadas('emissao_notas.csv')

In [16]:
contagem_cda = base_conjunta['cda'].value_counts()
registros_repetidos = contagem_cda[contagem_cda > 1]
df_registros_repetidos = registros_repetidos.reset_index()
df_registros_repetidos.columns = ['cda', 'contagem']
df_registros_repetidos

,cda,contagem


In [17]:
base_conjunta.columns

Index(['cda', 'tipo_divida', 'id_contribuinte', 'atividade_principal',
       'situacao', 'tipo_tributo', 'vlr_pago', 'valor_tot', 'vlr_tributo',
       'vlr_taxa', 'competencia_divida', 'inscricao_divida',
       'arrecadacao_divida', 'ajuizamento_divida', 'edificacao',
       'cpf_cnpj_existe', 'protesto', 'ajuizamento', 'refis', 'deb_totais',
       'deb_pagos', 'idade_divida', 'quantidade_reparcelamento'],
      dtype='object')

In [102]:
base_conjunta['data_divida'] = pd.to_datetime(base_conjunta['inscricao_divida'], infer_datetime_format = True)
base_conjunta['ano_inscricao_da'] = base_conjunta['data_divida'].dt.year

In [58]:
#status_situacao0 = base_imovel[base_imovel['da_aberto'] == 1]
status_situacao0 = base_conjunta[base_conjunta['da_aberto'] == 1]
status_situacao0 = status_situacao0[status_situacao0['anos_idade_da'] < 15]
status_situacao0 = status_situacao0[status_situacao0['cpf_cnpj_existe'] == 0]
status_situacao0 = formatar_moeda(status_situacao0['valor_tot'].sum())
status_situacao0

'R$ 478.880.439,14'

# 02) Criando variáveis para clusterização

In [103]:
# incorporei a variáve tipo_divida que antes não tinha

dados_pessoas = base_conjunta[['cda', 'id_contribuinte', 'situacao', 'cpf_cnpj_existe', 'edificacao', 'deb_totais', 'deb_pagos', 'valor_tot', 'vlr_pago', 'tipo_divida']]
dados_pessoas['id_contribuinte'] = dados_pessoas['id_contribuinte'].astype(str)  # Convertendo para string

dados_pessoas.dropna(subset=['id_contribuinte'], inplace=True)
dados_pessoas.rename(columns={'id_contribuinte': 'id_pessoa'}, inplace=True)



In [83]:
dados_pessoas.columns

Index(['cda', 'id_pessoa', 'situacao', 'cpf_cnpj_existe', 'edificacao',
       'deb_totais', 'deb_pagos', 'valor_tot', 'vlr_pago', 'tipo_divida'],
      dtype='object')

In [78]:
# # Renomeio a coluna criada anteriormente para valor_pago
# dados_pessoas.rename(columns={'valor_pago_vista_parc':'valor_pago'}, inplace=True)

In [104]:
# Calculo que apresenta quantas cdas o contribuinte tem

frequencia_da_pessoa = dados_pessoas.groupby(['id_pessoa'])['cda'].nunique()
total_debitos_pessoa = dados_pessoas.groupby(['id_pessoa'])['deb_totais'].sum()
debitos_pagos_pessoa = dados_pessoas.groupby(['id_pessoa'])['deb_pagos'].sum()
valor_total_pessoa = dados_pessoas.groupby(['id_pessoa'])['valor_tot'].sum()
valor_pago_pessoa = dados_pessoas.groupby(['id_pessoa'])['vlr_pago'].sum()

In [105]:
# Agrega informação da base de notas fiscais
dados_pessoas = pd.merge(
    left=dados_pessoas, left_on='id_pessoa', 
    right=base_notas_fiscais, right_on='id_pessoa', how='left'
)

In [106]:
# Verifica valores zerados

dados_pessoas.isnull().sum()

cda                      0
id_pessoa                0
situacao                 0
cpf_cnpj_existe          0
edificacao               0
deb_totais               0
deb_pagos                0
valor_tot                0
vlr_pago                 0
tipo_divida              0
qtd_notas_2anos    2278959
dtype: int64

In [107]:
# Substituindo por zero os valores nulos
# dados_pessoas['edificacao'] = dados_pessoas['edificacao'].fillna(0)
dados_pessoas['qtd_notas_2anos'] = dados_pessoas['qtd_notas_2anos'].fillna(0)

In [108]:
# Cria variável de situação do contribuinte tratando mercantil e imovel em suas respectivas variáveis ( DEBATER COM EQUIPE )

# MERCANTIL
dados_pessoas.loc[(dados_pessoas['tipo_divida'] == 'mercantil' ) & (dados_pessoas['qtd_notas_2anos'] > 0) & (dados_pessoas['situacao'] == 'ATIVO'), 'situacao_ativa'] = 2
dados_pessoas.loc[(dados_pessoas['tipo_divida'] == 'mercantil' ) & (dados_pessoas['qtd_notas_2anos'] > 0) & (dados_pessoas['situacao'] != 'ATIVO'), 'situacao_ativa'] = 1
dados_pessoas.loc[(dados_pessoas['tipo_divida'] == 'mercantil' ) & (dados_pessoas['qtd_notas_2anos'] == 0) & (dados_pessoas['situacao'] == 'ATIVO'), 'situacao_ativa'] = 1
dados_pessoas.loc[(dados_pessoas['tipo_divida'] == 'mercantil' ) & (dados_pessoas['qtd_notas_2anos'] == 0) & (dados_pessoas['situacao'] != 'ATIVO'), 'situacao_ativa'] = 0

# IMOVEL
dados_pessoas.loc[(dados_pessoas['tipo_divida'] == 'imovel' ) & (dados_pessoas['edificacao'] == 1), 'situacao_ativa'] = 2
dados_pessoas.loc[(dados_pessoas['tipo_divida'] == 'imovel' ) & (dados_pessoas['edificacao'] == 0), 'situacao_ativa'] = 1
dados_pessoas.loc[(dados_pessoas['tipo_divida'] == 'imovel' ) & (dados_pessoas['edificacao'] == 0) & (dados_pessoas['cpf_cnpj_existe'] == 0), 'situacao_ativa'] = 0

# dados_pessoas.loc[(dados_pessoas['tipo_divida'] == 'imovel' ) & (dados_pessoas['edificacao'] == 1) & (dados_pessoas['situacao'] == 'ATIVO'), 'situacao_ativa'] = 2
# dados_pessoas.loc[(dados_pessoas['tipo_divida'] == 'imovel' ) & (dados_pessoas['edificacao'] == 0) & (dados_pessoas['situacao'] == 'ATIVO'), 'situacao_ativa'] = 1
# dados_pessoas.loc[(dados_pessoas['tipo_divida'] == 'imovel' ) & (dados_pessoas['edificacao'] == 1) & (dados_pessoas['situacao'] != 'ATIVO'), 'situacao_ativa'] = 0
# dados_pessoas.loc[(dados_pessoas['tipo_divida'] == 'imovel' ) & (dados_pessoas['edificacao'] == 0) & (dados_pessoas['situacao'] != 'ATIVO'), 'situacao_ativa'] = 0

# o que é nulo colocamos peso 0 
dados_pessoas['situacao_ativa'] = dados_pessoas['situacao_ativa'].fillna(0)

In [ ]:
# status_situacao0 = dados_pessoas[dados_pessoas['situacao_ativa'] == 0]
# status_situacao0

In [22]:
# contagem_situacao = status_situacao0['situacao'].value_counts()
# contagem_situacao

INAPTO      711848
SUSPENSO    145524
BAIXADO      32470
Name: situacao, dtype: int64

In [23]:
# contagem_tipo_divida = status_situacao0['tipo_divida'].value_counts()
# contagem_tipo_divida

mercantil    889842
Name: tipo_divida, dtype: int64

In [109]:
# Cria variável com peso sobre a situação real tratando cpf

dados_pessoas['status_situacao'] = dados_pessoas['situacao_ativa'] + dados_pessoas['cpf_cnpj_existe'] 
dados_pessoas.loc[dados_pessoas['situacao_ativa'] == 0, 'status_situacao'] = 0
# dados_pessoas.loc[(dados_pessoas['cpf_cnpj_existe'] == 0 ), 'status_situacao'] = 0

In [ ]:
dados_pessoas

---

In [110]:
# Remove duplicatas dos dados finalizando o dataframe
dados_pessoas.drop_duplicates(subset=['id_pessoa'], inplace=True)
dados_pessoas = dados_pessoas.set_index('id_pessoa')

dados_pessoas['frequencia_da_pessoa'] = frequencia_da_pessoa
dados_pessoas['total_debitos_pessoa'] = total_debitos_pessoa
dados_pessoas['debitos_pagos_pessoa'] = debitos_pagos_pessoa
dados_pessoas['valor_total_pessoa'] = valor_total_pessoa
dados_pessoas['valor_pago_pessoa'] = valor_pago_pessoa

In [111]:
# Faz o calculo do historico de pagamento

dados_pessoas.loc[(dados_pessoas['total_debitos_pessoa'].isna()) | (dados_pessoas['total_debitos_pessoa'] == 0) , 'total_debitos_pessoa'] = 1
dados_pessoas.loc[(dados_pessoas['valor_total_pessoa'].isna()) | (dados_pessoas['valor_total_pessoa'] == 0) , 'valor_total_pessoa'] = 1

dados_pessoas['historico_pagamento_em_qtd'] = dados_pessoas['debitos_pagos_pessoa'] / (dados_pessoas['total_debitos_pessoa'])
dados_pessoas['historico_pagamento_em_valor'] = dados_pessoas['valor_pago_pessoa'] / (dados_pessoas['valor_total_pessoa'])

In [112]:
# Monta dataframe para clusterização

df_pipe_cluster = dados_pessoas.query("frequencia_da_pessoa > 1")
df_pipe_cluster = df_pipe_cluster[['status_situacao', 'historico_pagamento_em_qtd', 'historico_pagamento_em_valor']]

In [33]:
# df_pipe_cluster.loc[df_pipe_cluster['historico_pagamento_em_valor'] > 1, 'historico_pagamento_em_valor'] = 1

In [113]:
df_pipe_cluster.head()

,status_situacao,historico_pagamento_em_qtd,historico_pagamento_em_valor
id_pessoa,,,
ac28642d7c82b33f,3.0,0.000000,0.000000
fc4b99b807fbed41,0.0,0.000000,0.000000
96e8e553de69d7a4,0.0,0.000000,0.000000
6dbe14da38a31dc1,3.0,0.619718,0.115638
870c08c252b25ad1,3.0,0.622120,0.500272


In [114]:
# pipe
df_pipe_cluster.describe().T

,count,mean,std,min,25%,50%,75%,max
status_situacao,349837.0,1.504232,1.383882,0.0,0.0,2.0,3.000000,3.0
historico_pagamento_em_qtd,349837.0,0.255423,0.356742,0.0,0.0,0.0,0.500000,1.0
historico_pagamento_em_valor,349837.0,0.147998,0.263659,0.0,0.0,0.0,0.182127,1.0


# 03) Clusterização dos contribuintes

In [115]:
faixa_n_clusters = [i for i in range(2,16)]
valores_inercia = []
valores_score = []

for k in faixa_n_clusters:
    agrupador = KMeans(n_clusters=k, random_state=1337)
    label = agrupador.fit_predict(df_pipe_cluster)
    print(f"Treinamento do agrupador para K= {k} finalizado")
    
    media_inercia = agrupador.inertia_
    valores_inercia.append(media_inercia)
    print(f"Inércia calculada para o agrupador de K= {k}. Inércia: {media_inercia}")

    media_score = agrupador.score(df_pipe_cluster)
    valores_score.append(media_score)
    print(f"Score calculado para o agrupador de K= {k}. Socre: {media_score}")

Treinamento do agrupador para K= 2 finalizado
Inércia calculada para o agrupador de K= 2. Inércia: 101730.40682690097
Score calculado para o agrupador de K= 2. Socre: -101730.40682690099
Treinamento do agrupador para K= 3 finalizado
Inércia calculada para o agrupador de K= 3. Inércia: 58129.22074670863
Score calculado para o agrupador de K= 3. Socre: -58129.22074670863
Treinamento do agrupador para K= 4 finalizado
Inércia calculada para o agrupador de K= 4. Inércia: 31013.160943517018
Score calculado para o agrupador de K= 4. Socre: -31013.160943517018
Treinamento do agrupador para K= 5 finalizado
Inércia calculada para o agrupador de K= 5. Inércia: 21290.056745651524
Score calculado para o agrupador de K= 5. Socre: -21290.056745651524
Treinamento do agrupador para K= 6 finalizado
Inércia calculada para o agrupador de K= 6. Inércia: 14687.263602523557
Score calculado para o agrupador de K= 6. Socre: -14687.263602523555
Treinamento do agrupador para K= 7 finalizado
Inércia calculada par

In [116]:
fig =go.Figure()
fig.add_trace(go.Scatter(x = faixa_n_clusters, y = valores_inercia))
fig.update_layout(
    title="INDICADOR: Inercia para K grupos",
    xaxis_title="Valores de K",
    yaxis_title="Inércia",
    font=dict(
        family="Courier New, monospace",
        size=18,
        color="#7f7f7f"
    )
)
fig.show()

In [117]:
fig =go.Figure()
fig.add_trace(go.Scatter(x = faixa_n_clusters, y = valores_score))
fig.update_layout(
    title="Indicador: Erro quadratico médio para K grupos",
    xaxis_title="Valores de K",
    yaxis_title="MSE",
    font=dict(
        family="Courier New, monospace",
        size=18,
        color="#7f7f7f"
    )
)
fig.show()

In [118]:
def optimal_number_of_clusters(wcss):
    x1, y1 = 2, wcss[0]
    x2, y2 = 15, wcss[len(wcss)-1]

    distances = []
    for i in range(len(wcss)):
        x0 = i+2
        y0 = wcss[i]
        numerator = abs((y2-y1)*x0 - (x2-x1)*y0 + x2*y1 - y2*x1)
        denominator = np.sqrt((y2 - y1)**2 + (x2 - x1)**2)
        distances.append(numerator/denominator)
    
    return distances.index(max(distances)) + 2

In [119]:
valor_ideal_k = optimal_number_of_clusters(valores_inercia)

print("Melhor valor de K:", valor_ideal_k)

Melhor valor de K: 5


In [120]:
# Construindo o melhor agrupador de clusteres
valor_de_k = 4

agrupador = KMeans(n_clusters=valor_de_k, random_state=1337)
agrupador.fit_transform(df_pipe_cluster)

# Obtendo o ponto central dos clusteres
centros = agrupador.cluster_centers_
df_centroide = pd.DataFrame(centros, columns = df_pipe_cluster.columns).round(3)
df_centroide['cluster'] = df_centroide.index

# Obtendo o label para cada pessoa
df_pipe_cluster['label_cluster'] = agrupador.labels_

In [122]:
df_centroide

,status_situacao,historico_pagamento_em_qtd,historico_pagamento_em_valor,cluster
0,0.006,0.113,0.045,0
1,3.000,0.139,0.067,1
2,2.954,0.873,0.605,2
3,1.999,0.177,0.077,3


Na totalidade temos +1 cluster especial que é do contribuinte que apareceu a primiera vez em DA

In [123]:
dicionario_clusteres ={
    4: 'PRIMEIRA DIVIDA',
    0: 'PIOR PAGADOR',
    1: 'PAGADOR INTERMEDIARIO',
    3: 'BOM PAGADOR',
    2: 'MELHOR PAGADOR'
}

In [124]:
df_cluster = df_pipe_cluster.groupby('label_cluster')['label_cluster'].value_counts().to_frame()
total = df_cluster['label_cluster'].sum()
df_cluster['perc'] = df_cluster['label_cluster']/total
df_cluster

,,label_cluster,perc
label_cluster,label_cluster,,
0,0,155504,0.444504
1,1,81359,0.232563
2,2,57936,0.165609
3,3,55038,0.157325


Classificações dos contribuintes com base no melhor ao pior pagador, baseado no seu histórico e na sua situação atual.

# 04) Cria classificador de class do contribuinte

In [125]:
# Constroi o modelo que prevê qual o grupo do contribuinte

x_cluster = df_pipe_cluster.drop(columns=['label_cluster'])
y_cluster = df_pipe_cluster['label_cluster']

X_train, X_test, y_train, y_test = train_test_split(x_cluster, y_cluster, test_size=0.3, random_state=1337)

In [126]:
model_predict_contribuinte = RandomForestClassifier(random_state=1337)
model_predict_contribuinte.fit(X_train, y_train)

score_validacao = model_predict_contribuinte.score(X_test, y_test)
print("Score de validacao:", score_validacao)

Score de validacao: 0.9996665142160226


In [127]:
# Previsão da classificação para a base total de contribuintes

matriz_previsao_class = dados_pessoas[['status_situacao', 'historico_pagamento_em_qtd', 'historico_pagamento_em_valor']]
dados_pessoas['class_contribuinte'] = model_predict_contribuinte.predict(matriz_previsao_class)
dados_pessoas.loc[dados_pessoas['frequencia_da_pessoa'] == 1, 'class_contribuinte'] = 4


In [128]:
primeira_divida = dados_pessoas[dados_pessoas['frequencia_da_pessoa'] == 1]

# Como ficou a nossa classificação originalmente

df_primeira_divida = primeira_divida.groupby('class_contribuinte')['class_contribuinte'].value_counts().to_frame()#.reset_index()
df_primeira_divida.rename(columns = {'class_contribuinte':'num_contrib'}, inplace = True)
total = df_primeira_divida['num_contrib'].sum()
df_primeira_divida['perc'] = np.round(df_primeira_divida['num_contrib']/total, 3)
df_primeira_divida

,,num_contrib,perc
class_contribuinte,class_contribuinte,,
4,4,74222,1.0


In [129]:

# Como ficou a nossa classificação originalmente

df_descritiva = dados_pessoas.groupby('class_contribuinte')['class_contribuinte'].value_counts().to_frame()#.reset_index()
df_descritiva.rename(columns = {'class_contribuinte':'num_contrib'}, inplace = True)
total = df_descritiva['num_contrib'].sum()
df_descritiva['perc'] = np.round(df_descritiva['num_contrib']/total, 3)
df_descritiva

,,num_contrib,perc
class_contribuinte,class_contribuinte,,
0,0,155504,0.367
1,1,81357,0.192
2,2,57939,0.137
3,3,55037,0.130
4,4,74222,0.175


In [130]:
# Nomeando a classificação com label de prioridade
dados_pessoas.loc[dados_pessoas['class_contribuinte'] == 4, 'class_contribuinte_nome'] = 'PRIMEIRA DIVIDA'
dados_pessoas.loc[dados_pessoas['class_contribuinte'] == 0, 'class_contribuinte_nome'] = 'PIOR PAGADOR'
dados_pessoas.loc[dados_pessoas['class_contribuinte'] == 1, 'class_contribuinte_nome'] = 'PAGADOR INTERMEDIARIO'
dados_pessoas.loc[dados_pessoas['class_contribuinte'] == 3, 'class_contribuinte_nome'] = 'BOM PAGADOR'
dados_pessoas.loc[dados_pessoas['class_contribuinte'] == 2, 'class_contribuinte_nome'] = 'MELHOR PAGADOR'

In [131]:
df_classificao_contribuinte = dados_pessoas[['class_contribuinte_nome']].reset_index()

In [132]:
df_classificao_contribuinte

,id_pessoa,class_contribuinte_nome
0,ac28642d7c82b33f,PAGADOR INTERMEDIARIO
1,fc4b99b807fbed41,PIOR PAGADOR
2,96e8e553de69d7a4,PIOR PAGADOR
3,6dbe14da38a31dc1,PAGADOR INTERMEDIARIO
4,870c08c252b25ad1,MELHOR PAGADOR
...,...,...
424054,2fba3f9602ee622b,PRIMEIRA DIVIDA
424055,d6e7cb2581a47f25,PRIMEIRA DIVIDA
424056,89a2cb2e2e9475a8,PRIMEIRA DIVIDA
424057,22e610a1d7dc59f1,PRIMEIRA DIVIDA


In [133]:
base_conjunta.columns

Index(['cda', 'tipo_divida', 'id_contribuinte', 'atividade_principal',
       'situacao', 'tipo_tributo', 'vlr_pago', 'valor_tot', 'vlr_tributo',
       'vlr_taxa', 'competencia_divida', 'inscricao_divida',
       'arrecadacao_divida', 'ajuizamento_divida', 'edificacao',
       'cpf_cnpj_existe', 'protesto', 'ajuizamento', 'refis', 'deb_totais',
       'deb_pagos', 'idade_divida', 'quantidade_reparcelamento', 'data_divida',
       'ano_inscricao_da'],
      dtype='object')

# 05) Análise discriminante da classificação do contribuintes para o % Pago das dívidas de cada CDA

In [134]:
# Calcula variável target y
base_conjunta['percentual_pago_cda'] = base_conjunta['vlr_pago'] / base_conjunta['valor_tot']

In [135]:
base_conjunta[['cda', 'tipo_divida', 'vlr_pago', 'valor_tot', 'percentual_pago_cda' ]]

,cda,tipo_divida,vlr_pago,valor_tot,percentual_pago_cda
0,00000123c3d4731c6b3a8e268c80aedf1,imovel,0.0,2048.73,0.0
1,00000123c3d4731c6b3a8e268c80aedf2,mercantil,0.0,2515.85,0.0
2,00000b44c5ba1e669ceed47545e621dd2,mercantil,0.0,1278.97,0.0
3,000014e359592e62d8a3e5cebc255ca6E,mercantil,0.0,847.02,0.0
4,0000331f601a73e52b46f673bf0c61251,imovel,0.0,344.12,0.0
...,...,...,...,...,...
2414893,ffffd8af5760cada9fcacff532cf50c41,imovel,0.0,1728.13,0.0
2414894,ffffdb721b90282718565e26094f06ee2,mercantil,0.0,240.77,0.0
2414895,ffffdc78aa7a90e26b3a8e268c80aedf1,imovel,0.0,9390.43,0.0
2414896,fffff192f894983c9ceed47545e621dd1,imovel,0.0,16594.80,0.0


In [ ]:
# # Substitui por zero os valores nulos em que a divisão for zero
# base_conjunta['percentual_pago_cda'] = base_conjunta['percentual_pago_cda'].fillna(0)

In [72]:
base_conjunta.columns

Index(['cda', 'tipo_divida', 'id_contribuinte', 'atividade_principal',
       'situacao', 'tipo_tributo', 'vlr_pago', 'valor_tot', 'vlr_tributo',
       'vlr_taxa', 'competencia_divida', 'inscricao_divida',
       'arrecadacao_divida', 'ajuizamento_divida', 'edificacao',
       'cpf_cnpj_existe', 'protesto', 'ajuizamento', 'refis', 'deb_totais',
       'deb_pagos', 'idade_divida', 'quantidade_reparcelamento', 'data_divida',
       'ano_inscricao_da', 'percentual_pago_cda'],
      dtype='object')

In [136]:
df = base_conjunta[base_conjunta['percentual_pago_cda'].isna()]
df[['cda', 'tipo_divida', 'valor_tot', 'vlr_pago', 'vlr_tributo', 'vlr_taxa', 'idade_divida', 'inscricao_divida', 'percentual_pago_cda']]

,cda,tipo_divida,valor_tot,vlr_pago,vlr_tributo,vlr_taxa,idade_divida,inscricao_divida,percentual_pago_cda
1840081,c321f72f88d3f46d9ceed47545e621dd1,imovel,0.0,0.0,0.0,0.0,18.0,2005-09-30,NaN


In [143]:
base_conjunta['percentual_pago_cda'] = base_conjunta['percentual_pago_cda'].fillna(0)

In [144]:
df2 = base_conjunta[base_conjunta['percentual_pago_cda'] == np.inf]
df2[['cda', 'tipo_divida', 'valor_tot', 'vlr_pago', 'vlr_tributo', 'vlr_taxa', 'idade_divida', 'inscricao_divida', 'percentual_pago_cda']]
# df2 = formatar_moeda(df2['vlr_tributo'].sum())
df2

,cda,tipo_divida,id_contribuinte,atividade_principal,situacao,tipo_tributo,vlr_pago,valor_tot,vlr_tributo,vlr_taxa,...,protesto,ajuizamento,refis,deb_totais,deb_pagos,idade_divida,quantidade_reparcelamento,data_divida,ano_inscricao_da,percentual_pago_cda


In [145]:
# Separação dos dados para a análise discriminante dos grupos de contribuintes

df_analise_discriminante = pd.merge(
    left=base_conjunta, left_on='id_contribuinte', right=df_classificao_contribuinte, right_on='id_pessoa', how='left'
)

df_analise_discriminante = df_analise_discriminante[['percentual_pago_cda', 'class_contribuinte_nome']]

In [146]:
df_analise_discriminante.groupby("class_contribuinte_nome").describe().T

class_contribuinte_nome      BOM PAGADOR  MELHOR PAGADOR  \
percentual_pago_cda count  439688.000000   276703.000000   
                    mean        0.076223        0.603698   
                    std         0.231124        0.399810   
                    min         0.000000        0.000000   
                    25%         0.000000        0.231267   
                    50%         0.000000        0.654910   
                    75%         0.000000        1.000000   
                    max         1.000000        1.000000   

class_contribuinte_nome    PAGADOR INTERMEDIARIO   PIOR PAGADOR  \
percentual_pago_cda count          674495.000000  949790.000000   
                    mean                0.093626       0.050595   
                    std                 0.254588       0.196850   
                    min                 0.000000       0.000000   
                    25%                 0.000000       0.000000   
                    50%                 0.000000       0.000000   
                    75%                 0.000000       0.000000   
                    max                 1.000000       1.000000   

class_contribuinte_nome    PRIMEIRA DIVIDA  
percentual_pago_cda count     74222.000000  
                    mean          0.351527  
                    std           0.439288  
                    min           0.000000  
                    25%           0.000000  
                    50%           0.000000  
                    75%           1.000000  
                    max           1.000000

In [147]:
# Dummyzando a variável de classificação 

ohe = OneHotEncoder(dtype=int)

colunas_ohe = ohe.fit_transform(df_analise_discriminante[['class_contribuinte_nome']]).toarray()
df_2 = pd.DataFrame(colunas_ohe, columns=ohe.get_feature_names_out(['class_contribuinte_nome']))

df_n_categorico = df_analise_discriminante.drop(columns=['class_contribuinte_nome'], axis=1)
df_pipe_discriminante = pd.concat([df_n_categorico, df_2], axis=1)

In [148]:
x_analise_discriminante = df_pipe_discriminante.drop(columns=['percentual_pago_cda'])
y_analise_discriminante = df_pipe_discriminante['percentual_pago_cda'].astype('int')

In [149]:
analise_discriminante = LinearDiscriminantAnalysis()
analise_discriminante.fit(x_analise_discriminante, y_analise_discriminante)

LinearDiscriminantAnalysis()

In [150]:
dados_analise_disc = {'variavel': analise_discriminante.feature_names_in_, 'coeficiente' : analise_discriminante.coef_[0].round(5)}
pesos_analise_disc = pd.DataFrame(dados_analise_disc).sort_values('variavel').reset_index().drop(columns=['index'])

In [151]:
pesos_analise_disc

,variavel,coeficiente
0,class_contribuinte_nome_BOM PAGADOR,-0.81024
1,class_contribuinte_nome_MELHOR PAGADOR,4.68032
2,class_contribuinte_nome_PAGADOR INTERMEDIARIO,-0.60632
3,class_contribuinte_nome_PIOR PAGADOR,-0.98031
4,class_contribuinte_nome_PRIMEIRA DIVIDA,2.54487


Conclusão: O modelo pode se utilizar do fator de multiplicação do rating do contribuinte para identificar O melhor pagador, o pior pagador, e a primeira dívida. Os demais são intermediários e terão o mesmo peso de atribuição. 

**Como fica a modelagem da regressão**

Variável target:
- % pago da dívida ativa

Variáveis explicativas: 
- Rating do contribuinte baseado no Cluster encontrado
- Idade da dívida em anos
- % De pagamento dos processos de parcelamento (em valor)
- Quantidade de reparcelamentos
- Tipo de dívida

# 06) Salva o modelo de classificação dos contribuintes

In [152]:
def salva_modelo_serializado(nome_modelo_serializado, modelo):
    sav_best_model = open(nome_modelo_serializado, 'wb')
    pickle.dump(modelo, sav_best_model)
    sav_best_model.close()

    #os.makedirs(modelsPath) # nao entendi e tive que colocar isso pra abrir um novo diretorio
    pathModelo = modelsPath+"\\"+os.path.join(nome_modelo_serializado)
    shutil.move(os.path.abspath(nome_modelo_serializado), pathModelo)

In [153]:
salva_modelo_serializado("classificador-contribuinte-v2.pkl", model_predict_contribuinte)